#**SEP 775 Assignment - 4**
                        
**Student: Rutvik Roy**\
**Student ID: 400490159**


In [1]:
!pip install datasets
!pip install pandas
!pip install peft
!pip install tensorboard
!pip install torch
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [2]:
!pip install accelerate>=0.21.0
!pip install rouge

In [3]:
# import all necessary libraries

import torch
from datasets import load_dataset,load_metric,Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# Data Cleaning

In [4]:
# load dataset

dataset=load_dataset("flytech/python-codes-25k",split="train[:100]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
#dataset to dataframe

dataset_df = pd.DataFrame(dataset)
dataset_df.head()

,output,text,input,instruction
0,```python\ntasks = []\nwhile True:\n task =...,Help me set up my daily to-do list! Setting up...,Setting up your daily to-do list...,Help me set up my daily to-do list!
1,```python\nshopping_list = {}\nwhile True:\n ...,Create a shopping list based on my inputs! Cre...,Creating a shopping list...,Create a shopping list based on my inputs!
2,"```python\ntotal_time = 0\nfor i in range(1, 8...",Calculate how much time I spend on my phone pe...,Calculating weekly phone usage...,Calculate how much time I spend on my phone pe...
3,```python\ntotal_bill = float(input('Enter the...,Help me split the bill among my friends! Split...,Splitting the bill...,Help me split the bill among my friends!
4,```python\nmovie_list = {}\nwhile True:\n g...,Organize my movie list into genres! Organizing...,Organizing your movie list...,Organize my movie list into genres!


In [6]:
# remove (python\n) from the 'output' column

dataset_df['output'] = dataset_df['output'].str.replace("python\n", "")

# remove unnecessary string like quatations
dataset_df['output'] = dataset_df['output'].str.replace('```', '')

# keep only necessary columns only
dataset_df=dataset_df[['instruction','output']]

# remove duplicate entry
dataset_df.drop_duplicates(['instruction','output'],inplace=True)

# delete empty rows
dataset_df.dropna(inplace=True)

# select only 100 rows for less training time
dataset_df=dataset_df[:100]

dataset_df.head()

,instruction,output
0,Help me set up my daily to-do list!,tasks = []\nwhile True:\n task = input('Ent...
1,Create a shopping list based on my inputs!,shopping_list = {}\nwhile True:\n item = in...
2,Calculate how much time I spend on my phone pe...,"total_time = 0\nfor i in range(1, 8):\n tim..."
3,Help me split the bill among my friends!,total_bill = float(input('Enter the total bill...
4,Organize my movie list into genres!,movie_list = {}\nwhile True:\n genre = inpu...


In [7]:
input_text=dataset_df.instruction.values
output_code=dataset_df.output.values

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# the tokenizer of pretained model T5-base
model_name = 'Salesforce/codet5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

In [9]:
def tokenize_text(text):
  return tokenizer.tokenize(text)

def tokenized_text_id(tokenized_text):
  return tokenizer.convert_tokens_to_ids(tokenized_text)

def add_special_tokens(input_text):
  return ['<CLS>']+input_text+['<SEP>']

In [10]:
print("Original:",input_text[0])
print("Tokenized input:",tokenizer.tokenize(input_text[0]))
print("Tokenized input ids:",tokenizer.convert_tokens_to_ids(tokenizer.tokenize(input_text[0])))
print(" ")
print("Original:",output_code[0])
print("Tokenized output:",tokenizer.tokenize(output_code[0]))
print("Tokenized output ids:",tokenizer.convert_tokens_to_ids(tokenizer.tokenize(output_code[0])))

Original: Help me set up my daily to-do list!
Tokenized input: ['Help', 'Ġme', 'Ġset', 'Ġup', 'Ġmy', 'Ġdaily', 'Ġto', '-', 'do', 'Ġlist', '!']
Tokenized input ids: [6696, 1791, 444, 731, 3399, 18872, 358, 17, 2896, 666, 5]
 
Original: tasks = []
while True:
    task = input('Enter a task or type 'done' to finish: ')
    if task == 'done': break
    tasks.append(task)
print(f'Your to-do list for today: {tasks}')

Tokenized output: ['tasks', 'Ġ=', 'Ġ[]', 'Ċ', 'while', 'ĠTrue', ':', 'Ċ', 'ĠĠĠ', 'Ġtask', 'Ġ=', 'Ġinput', "('", 'Enter', 'Ġa', 'Ġtask', 'Ġor', 'Ġtype', "Ġ'", 'done', "'", 'Ġto', 'Ġfinish', ':', "Ġ')", 'Ċ', 'ĠĠĠ', 'Ġif', 'Ġtask', 'Ġ==', "Ġ'", 'done', "':", 'Ġbreak', 'Ċ', 'ĠĠĠ', 'Ġtasks', '.', 'append', '(', 'task', ')', 'Ċ', 'print', '(', 'f', "'", 'Your', 'Ġto', '-', 'do', 'Ġlist', 'Ġfor', 'Ġtoday', ':', 'Ġ{', 'tasks', "}'", ')', 'Ċ']
Tokenized output ids: [9416, 273, 5378, 203, 17523, 1053, 30, 203, 565, 1562, 273, 810, 2668, 10237, 279, 1562, 578, 618, 296, 8734, 11, 358, 407

In [11]:
input_ids=[]

for input_txt in (input_text):
    input_txt=tokenize_text(input_txt)
    input_tokens=add_special_tokens(input_txt)
    ids=tokenized_text_id(input_tokens)
    input_ids.append(ids)

output_ids=[]

for output in (output_code):
    output=tokenize_text(output)
    output_tokens=add_special_tokens(output)
    ids=tokenized_text_id(output_tokens)
    output_ids.append(ids)

In [12]:
input_legnths=[]
for ids in input_ids:
    input_legnths.append(len(ids))
input_maximum_length=max(input_legnths)
print("Maximum input length:",input_maximum_length)


output_legnths=[]
for ids in output_ids:
    output_legnths.append(len(ids))
output_maximum_length=max(output_legnths)
print("Maximum output length:",output_maximum_length)

Maximum input length: 19
Maximum output length: 166


In [13]:
# function to tokenize the data and do padding according to maximum length

def tokenize_function(examples):
    inputs = tokenizer(examples['instruction'], padding='max_length', truncation=True, max_length=19, return_tensors='pt')
    inputs['labels'] = tokenizer(examples['output'], padding='max_length', truncation=True, max_length=166, return_tensors='pt')['input_ids']
    return inputs

In [14]:
# Split the dataset into training, validation, and test sets

train_df, temp_df = train_test_split(dataset_df, train_size=0.8, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# **Impliment Baseline Model and LORA**

In [15]:
from peft import PeftConfig, get_peft_model, LoraConfig, TaskType
from transformers import AutoModelForSeq2SeqLM

model_name = 'Salesforce/codet5-base'
model=AutoModelForSeq2SeqLM.from_pretrained(model_name)

# freez the original or baseline model parameters
for param in model.parameters():
    param.requires_grad = False


config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [16]:
# LORA configuration to be adapted

peft_config=LoraConfig(

    task_type=TaskType.SEQ_2_SEQ_LM,

    # the dimension of the low-rank matrices
    r=4,

    # the scaling factor for the low-rank matrices
    lora_alpha=32,

    # the dropout probability of the LoRA layers
    lora_dropout=0.01,

    target_modules=["k","q","v","o"],
)

peft_model=get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 884,736 || all params: 223,766,784 || trainable%: 0.3953830788397978


In [17]:
# verify trainable parameters using require_grad method

trainable_params = 0
all_param = 0

for _, param in peft_model.named_parameters():

    #adding parameter
    all_param += param.numel()
    
    #adding parameters to trainable if they require a graident
    if param.requires_grad:
        trainable_params += param.numel()


print(f"trainable params: {trainable_params}")
print(f"all params: {all_param}")
print(f"trainable: {100 * trainable_params / all_param:.2f}%")

trainable params: 884736
all params: 223766784
trainable: 0.40%


In [18]:
# the data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=peft_model)

# dataset object for training
training_data = Dataset.from_pandas(train_df)

# tokenize the training data
tokenized_training_data = training_data.map(tokenize_function, batched=True)

# dataset object for validation
validation_data = Dataset.from_pandas(val_df)

# tokenize the validation data
tokenized_validation_data = validation_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [19]:
training_data

Dataset({
    features: ['instruction', 'output', '__index_level_0__'],
    num_rows: 40
})

In [20]:
tokenized_training_data[:1]

{'instruction': ['Calculate my carbon footprint based on my daily activities!'],
 'output': ["transport = input('How do you usually commute (car, bike, walk, public)? ')\nfood = input('Are you a vegetarian, vegan, or omnivore? ')\nelectricity = float(input('Approximate monthly electricity bill: '))\ncarbon_footprint = {'car': 2.4, 'bike': 0.1, 'walk': 0, 'public': 1}[transport] + {'vegetarian': 1, 'vegan': 0.7, 'omnivore': 1.5}[food] + (electricity * 0.001)\nprint(f'Your estimated daily carbon footprint is {carbon_footprint} kg.')\n"],
 '__index_level_0__': [12],
 'input_ids': [[1,
   8695,
   3399,
   28755,
   27556,
   2511,
   603,
   3399,
   18872,
   17259,
   5,
   2,
   0,
   0,
   0,
   0,
   0,
   0,
   0]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]],
 'labels': [[1,
   13049,
   273,
   810,
   2668,
   44,
   543,
   741,
   1846,
   11234,
   1543,
   624,
   261,
   9815,
   16,
   324,
   2547,
   16,
   5442,
   16,
   1071,
   9945,

In [21]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

output_dir="./LORA_model"

# training args

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
	per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=100,
    learning_rate=0.001,
    num_train_epochs=100,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=100,
    report_to="tensorboard",
    load_best_model_at_end=False,
    save_strategy="no",
)

# trainer instance

trainer = Seq2SeqTrainer(
    model=peft_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_training_data,
    eval_dataset=tokenized_validation_data,)

model.config.use_cache = False  # silence the warnings.

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [22]:
# start training

trainer.train()

Step,Training Loss,Validation Loss
100,0.884200,0.820719
200,0.123800,0.920829
300,0.041200,0.967752
400,0.022900,1.004380
500,0.016300,1.017744


TrainOutput(global_step=500, training_loss=0.2176723017692566, metrics={'train_runtime': 183.8049, 'train_samples_per_second': 21.762, 'train_steps_per_second': 2.72, 'total_flos': 90795626496000.0, 'train_loss': 0.2176723017692566, 'epoch': 100.0})

In [23]:
#save trained model

import os
output_dir="/content/LORA_model"

if not os.path.exists(output_dir):
  os.makedirs(output_dir)
  
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/content/LORA_model/tokenizer_config.json',
 '/content/LORA_model/special_tokens_map.json',
 '/content/LORA_model/vocab.json',
 '/content/LORA_model/merges.txt',
 '/content/LORA_model/added_tokens.json',
 '/content/LORA_model/tokenizer.json')

# **Model Evaluation**

In [24]:
from peft import PeftModel, PeftConfig

# load peft config for pre-trained checkpoint

peft_model_id="/content/LORA_model"
config = PeftConfig.from_pretrained(peft_model_id)

# load model and tokenizer

trained_model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

trained_model = PeftModel.from_pretrained(trained_model, peft_model_id)
trained_model.eval()

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32100, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32100, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.01, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=4, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=4, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
              

In [25]:
# Testing with Prompts (these prompts or instructions have been taken from original flytech/python-25k training dataset)

prompts = ['Help me set up my daily to-do list!', 'Create a shopping list based on my inputs!', 'Calculate how much time I spend on my phone per week!', "Help me split the bill among my friends!"]
generated_code_list_1 = []

# Generate code for each prompt

for prompt in prompts:
    input_ids = tokenizer(prompt, return_tensors="pt", padding=True, max_length=19, truncation=True).input_ids.to(trained_model.device)
    generated_ids = trained_model.generate(input_ids=input_ids, max_length=166)
    generated_code = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    generated_code_list_1.append(generated_code)

In [26]:
print(generated_code_list_1)

["tasks = []\nwhile True:\n    task = input('Enter a task or type 'done' to finish: ')\n    if task == 'done': break\n    tasks.append(task)\nprint(f'Your to-do list for today: {tasks}')\n", "shopping_list = {}\nwhile True:\n    item = input('Enter an item or type 'done' to finish: ')\n    if item == 'done': break\n    quantity = input(f'Enter the quantity for {item}: ')\n    shopping_list[item] = quantity\nprint(f'Your shopping list: {shopping_list}')\n", "total_time = 0\nfor i in range(1, 8):\n    time = float(input(f'Enter phone usage in hours for day {i}: '))\n    total_time += time\nprint(f'You spend approximately {total_time} hours per week on your phone.')\n", "total_bill = float(input('Enter the total bill amount: '))\nfriends_count = int(input('Enter the number of friends: '))\nper_person = total_bill / friends_count\nprint(f'Each person should pay {per_person}')\n"]


# 1. BLUE Score

In [27]:
from nltk.translate.bleu_score import sentence_bleu,SmoothingFunction

# reference code or true code for the prompts
reference_code =["tasks = []\nwhile True:\n    task = input('Enter a task or type 'done' to finish: ')\n    if task == 'done': break\n    tasks.append(task)\nprint(f'Your to-do list for today: {tasks}')\n",
                 "shopping_list = {}\nwhile True:\n    item = input('Enter an item or type 'done' to finish: ')\n    if item == 'done': break\n    quantity = input(f'Enter the quantity for {item}: ')\n    shopping_list[item] = quantity\nprint(f'Your shopping list: {shopping_list}')\n",
                 "total_time = 0\nfor i in range(1, 8):\n    time = float(input(f'Enter phone usage in hours for day {i}: '))\n    total_time += time\nprint(f'You spend approximately {total_time} hours per week on your phone.')\n",
                 "total_bill = float(input('Enter the total bill amount: '))\nfriends_count = int(input('Enter the number of friends: '))\nper_person = total_bill / friends_count\nprint(f'Each person should pay {per_person}')\n"]
bleu_scores_1 = []

# calculate BLEU scores for each prompt

for generated, reference in zip(generated_code_list_1, reference_code):
    generated_tokens = generated.split()
    reference_tokens = reference.split()
    bleu_score = sentence_bleu([reference_tokens], generated_tokens)
    bleu_scores_1.append(bleu_score)

# print BLEU scores for each prompt
for i, (prompt, bleu_score,code) in enumerate(zip(prompts, bleu_scores_1,generated_code_list_1)):
    print(f"Prompt {i+1}: {prompt}")
    print(f"Generated Code: {code}")
    print(f"BLEU Score: {bleu_score:.4f}")
    print("-" * 50)

# calculate the average BLEU score
average_bleu_score = sum(bleu_scores_1) / len(bleu_scores_1)
print("Average BLEU Score of Lora model:", average_bleu_score)

Prompt 1: Help me set up my daily to-do list!
Generated Code: tasks = []
while True:
    task = input('Enter a task or type 'done' to finish: ')
    if task == 'done': break
    tasks.append(task)
print(f'Your to-do list for today: {tasks}')

BLEU Score: 1.0000
--------------------------------------------------
Prompt 2: Create a shopping list based on my inputs!
Generated Code: shopping_list = {}
while True:
    item = input('Enter an item or type 'done' to finish: ')
    if item == 'done': break
    quantity = input(f'Enter the quantity for {item}: ')
    shopping_list[item] = quantity
print(f'Your shopping list: {shopping_list}')

BLEU Score: 1.0000
--------------------------------------------------
Prompt 3: Calculate how much time I spend on my phone per week!
Generated Code: total_time = 0
for i in range(1, 8):
    time = float(input(f'Enter phone usage in hours for day {i}: '))
    total_time += time
print(f'You spend approximately {total_time} hours per week on your phone.')

B

# 2. ROUGE Score

In [28]:
from rouge import Rouge

rouge=Rouge()
rouge_scores_1 = []

print(f"Rouge Scores of Lora Model:")
print(" ")

# calculate Rouge scores for each prompt
for generated, reference in zip(generated_code_list_1, reference_code):
    rouge_score = rouge.get_scores([generated], [reference])
    rouge_scores_1.append(rouge_score)

# print Rouge scores for each prompt
for i, (prompt, rouge_score) in enumerate(zip(prompts, rouge_scores_1)):
    print(f"Prompt {i+1}: {prompt}")
    print(f"Rouge Score:",rouge_score)
    print("-" * 50)

Rouge Scores of Lora Model:
 
Prompt 1: Help me set up my daily to-do list!
Rouge Score: [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}]
--------------------------------------------------
Prompt 2: Create a shopping list based on my inputs!
Rouge Score: [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}]
--------------------------------------------------
Prompt 3: Calculate how much time I spend on my phone per week!
Rouge Score: [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}]
--------------------------------------------------
Prompt 4: Help me split the bill among my friends!
Rouge Score: [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0

# **Comparision with baseline model**

In [29]:
# load the baseline model which has been used for training

model_name = "Salesforce/codet5-base"
baseline_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [30]:
generated_code_list_2 = []

# Generate code for each prompt using baseline model

for prompt in prompts:
    input_ids = tokenizer(prompt, return_tensors="pt", padding=True, max_length=38, truncation=True).input_ids.to(baseline_model.device)
    generated_ids = baseline_model.generate(input_ids=input_ids,max_new_tokens=400)
    generated_code = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    generated_code_list_2.append(generated_code)

# 1. Baseline model BLUE score

In [31]:
from nltk.translate.bleu_score import sentence_bleu

bleu_scores_2 = []

# calculate BLEU scores for each prompt
for generated, reference in zip(generated_code_list_2, reference_code):
    generated_tokens = generated.split()
    reference_tokens = reference.split()
    bleu_score = sentence_bleu([reference_tokens], generated_tokens)
    bleu_scores_2.append(bleu_score)

# print BLEU scores for each prompt
for i, (prompt, bleu_score,code) in enumerate(zip(prompts, bleu_scores_2,generated_code_list_2)):
    print(f"Prompt {i+1}: {prompt}")
    print(f"Generated Code: {code}")
    print(f"BLEU Score: {bleu_score:.4f}")
    print("-" * 50)

# calculate the average BLEU score
average_bleu_score = sum(bleu_scores_2) / len(bleu_scores_2)
print("Average BLEU Score of Baseline model:", average_bleu_score)

Prompt 1: Help me set up my daily to-do list!
Generated Code: I'm not sure how to
BLEU Score: 0.0000
--------------------------------------------------
Prompt 2: Create a shopping list based on my inputs!
Generated Code:  function ( ) {
BLEU Score: 0.0000
--------------------------------------------------
Prompt 3: Calculate how much time I spend on my phone per week!
Generated Code:  public function calculateTime (
BLEU Score: 0.0000
--------------------------------------------------
Prompt 4: Help me split the bill among my friends!
Generated Code:  public class Bill {
BLEU Score: 0.0000
--------------------------------------------------
Average BLEU Score of Baseline model: 3.061619493425024e-234


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

# 2. Baseline model Rouge score

In [32]:
from rouge import Rouge

rouge=Rouge()
rouge_scores_2 = []

print(f"Rouge Scores of Baseline Model:")
print(" ")

# calculate Rouge scores for each prompt
for generated, reference in zip(generated_code_list_2, reference_code):
    rouge_score = rouge.get_scores([generated], [reference])
    rouge_scores_2.append(rouge_score)

# print Rouge scores for each prompt
for i, (prompt, rouge_score) in enumerate(zip(prompts, rouge_scores_2)):
    print(f"Prompt {i+1}: {prompt}")
    print(f"Rouge Score:",rouge_score)
    print("-" * 50)

Rouge Scores of Baseline Model:
 
Prompt 1: Help me set up my daily to-do list!
Rouge Score: [{'rouge-1': {'r': 0.04, 'p': 0.2, 'f': 0.066666663888889}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.04, 'p': 0.2, 'f': 0.066666663888889}}]
--------------------------------------------------
Prompt 2: Create a shopping list based on my inputs!
Rouge Score: [{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}]
--------------------------------------------------
Prompt 3: Calculate how much time I spend on my phone per week!
Rouge Score: [{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}]
--------------------------------------------------
Prompt 4: Help me split the bill among my friends!
Rouge Score: [{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, '